# Assignment 2: Milestone I Natural Language Processing
## Task 1. Basic Text Pre-processing

Version: 1.0

Environment: Python 3 and Jupyter notebook

Main Libraries used in Task 1:
* pandas
* re
* numpy
* datetime
* nltk
* seaborn
* matplotlib

## Introduction to Task 1: Basic Text Pre-processing

Task 1 involves essential text preprocessing for a dataset of 750 job advertisements. Each ad, categorised by job type, contains crucial information within its description. This task encompasses tokenisation, lowercase conversion, removal of short words, stopwords, rare terms, and frequent words. The resulting clean data will be used to construct a vocabulary stored in "vocab.txt," facilitating subsequent analysis. Adherence to specific formatting rules, including alphabetical word sorting and starting index at 0, is vital. Task 1 acts as a critical preparatory step, enabling meaningful insights and analysis of job advertisements.

## Importing libraries 

In [1]:
# Code to import libraries as you need in this assessment

# Data cleaning and tidying libraries
import numpy as np #linear algebra
import pandas as pd #datapreprocessing, CSV file I/O
from datetime import datetime

# Regular Expression Library
import re

# directory
import os

# NLP libraries
from nltk import RegexpTokenizer #tokenize words
from nltk.tokenize import sent_tokenize #tokenize sentences
from itertools import chain #to flatten list of lists

# Data visualisation libraries
import seaborn as sns #for plotting graphs
import matplotlib.pyplot as plt 

# Display DataFrame without scientific notation
pd.set_option('display.float_format', '{:.6f}'.format)

# Display all columns and rows
pd.set_option('display.max_columns', None)

## Task 1: Basic Text Pre-Processing (5 Marks)
- Examine the data folder, including the categories and job advertisment txt documents, etc. Explain your findings here, e.g., number of folders and format of txt files, etc.
- Load the data into proper data structures and get it ready for processing.
- Extract webIndex and description into proper data structures.

Output: vocab.txt

#### 1.0 Read in data

In [2]:
folder_path = 'data'
data_list = []

for subdir in os.listdir(folder_path):
    subfolder = os.path.join(folder_path, subdir)
    if os.path.isdir(subfolder):
        for file_name in os.listdir(subfolder):
            if file_name.endswith('.txt'):
                file_path = os.path.join(subfolder, file_name)
                with open(file_path, 'r') as file:
                    file_contents = file.read()

                    # Split the file contents into lines
                    lines = file_contents.split('\n')

                    # Create a dictionary to store the extracted fields and values
                    data_dict = {}

                    # Add 'job_category' key-value pair. this is used for classification model later on. 
                    data_dict['job_category'] = subdir

                    # Add file name
                    data_dict['file_name'] = file_name

                    for line in lines:
                        # Find the first occurrence of ':'
                        first_colon_index = line.find(':')
                        if first_colon_index != -1:
                            key = line[:first_colon_index].strip()
                            value = line[first_colon_index + 1:].strip()
                            data_dict[key] = value

                    # Append the dictionary to the data_list
                    data_list.append(data_dict)

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(data_list)
df.head()

,job_category,file_name,Title,Webindex,Company,Description
0,Sales,Job_00776.txt,Estate Agency Senior Sales Negotiator,72444142,ESTATE AGENCY PROFESSIONALS,Senior Sales Negotiator Location : Shoreditch ...
1,Sales,Job_00762.txt,Export Sales Executive (French & German Fluency),68687567,Next Recruitment Ltd,Previous experience in a similar role is essen...
2,Sales,Job_00763.txt,GRADUATE SALES ENGINEER,68257980,BMS Sales Specialists LLP,GRADUATE SALES ENGINEER Sector: Engineering Co...
3,Sales,Job_00749.txt,Sales Representative / Lead Generator,71168766,BlueTown Online,Job Title: Sales Representative / Lead Generat...
4,Sales,Job_00761.txt,Search Recruitment Consultant Media and Techn...,72441930,Fresh Partnership,Search Recruitment Consultant Media and Techn...


In [3]:
# Check number of jobs per category. cross this against original folder counts. 
df['job_category'].value_counts()

Engineering           231
Healthcare_Nursing    198
Accounting_Finance    191
Sales                 156
Name: job_category, dtype: int64

In [4]:
df['Webindex'].nunique()
# there are 776 unique job ads based on the Webindex column as a unique identifier.

776

In [5]:
# check null values
df.isnull().sum()
# We can see that there are 4 fields, and only NULL values where there is company. All other fields have values.

job_category     0
file_name        0
Title            0
Webindex         0
Company         89
Description      0
dtype: int64

#### 1.1 Extract information from each job advertisement

In [6]:
# Extract the webindex and description column and save into a different df called df_desc. Set the webindex as the index. ** remove webindex as index**
#df_desc = df[['Webindex','Description']].set_index('Webindex')
df_desc = df[['Webindex','Description']]
df_desc.head()

,Webindex,Description
0,72444142,Senior Sales Negotiator Location : Shoreditch ...
1,68687567,Previous experience in a similar role is essen...
2,68257980,GRADUATE SALES ENGINEER Sector: Engineering Co...
3,71168766,Job Title: Sales Representative / Lead Generat...
4,72441930,Search Recruitment Consultant Media and Techn...


#### 1.2 Convert to lower case

In [7]:
# convert description column to lower case and save into the same column
df_desc['Description'] = df_desc['Description'].str.lower()
df_desc.head()

/var/folders/hn/8z124v9x1x53hqvr0c989hsr0000gn/T/ipykernel_3271/2001018643.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_desc['Description'] = df_desc['Description'].str.lower()


,Webindex,Description
0,72444142,senior sales negotiator location : shoreditch ...
1,68687567,previous experience in a similar role is essen...
2,68257980,graduate sales engineer sector: engineering co...
3,71168766,job title: sales representative / lead generat...
4,72441930,search recruitment consultant media and techn...


#### 1.3 Tokenise each job advertisement

In [8]:
pattern = r"[a-zA-Z]+(?:[-'][a-zA-Z]+)?"
# this pattern can be explained by the below:
# [a-zA-Z]+ = any letter, upper or lower case, one or more times
# (?: = start of non-capturing group
# [-'] = hyphen or apostrophe
# [a-zA-Z]+ = any letter, upper or lower case, one or more times
# )? = end of non-capturing group, zero or one times

# Tokenize the description column
df_desc_token = df_desc['Description'].apply(lambda x: RegexpTokenizer(pattern).tokenize(x))


In [9]:
df_desc.head()

,Webindex,Description
0,72444142,senior sales negotiator location : shoreditch ...
1,68687567,previous experience in a similar role is essen...
2,68257980,graduate sales engineer sector: engineering co...
3,71168766,job title: sales representative / lead generat...
4,72441930,search recruitment consultant media and techn...


In [10]:
df_desc_token.head()

0    [senior, sales, negotiator, location, shoredit...
1    [previous, experience, in, a, similar, role, i...
2    [graduate, sales, engineer, sector, engineerin...
3    [job, title, sales, representative, lead, gene...
4    [search, recruitment, consultant, media, and, ...
Name: Description, dtype: object

In [11]:
# strip white space
df_desc_token = df_desc_token.apply(lambda x: [token.strip() for token in x])
df_desc_token.head()

0    [senior, sales, negotiator, location, shoredit...
1    [previous, experience, in, a, similar, role, i...
2    [graduate, sales, engineer, sector, engineerin...
3    [job, title, sales, representative, lead, gene...
4    [search, recruitment, consultant, media, and, ...
Name: Description, dtype: object

In [12]:
#stats function from Week 7 Lab
def stats_print(tk_reviews):
    """
    This function takes a list of tokenised reviews as input and prints the following statistics:
    - Vocabulary size: number of unique tokens
    - Total number of tokens: number of tokens in the corpus
    - Lexical diversity: vocabulary size divided by total number of tokens
    - Total number of reviews: number of reviews in the corpus
    - Average review length: average number of tokens per review
    - Maximum review length: maximum number of tokens in a review
    - Minimum review length: minimum number of tokens in a review
    - Standard deviation of review length: standard deviation of the number of tokens in a review
    """
    words = list(chain.from_iterable(tk_reviews)) # we put all the tokens in the corpus in a single list
    vocab = set(words) # compute the vocabulary by converting the list of words/tokens to a set, i.e., giving a set of unique words
    lexical_diversity = len(vocab)/len(words)
    print("Vocabulary size: ",len(vocab))
    print("Total number of tokens: ", len(words))
    print("Lexical diversity: ", lexical_diversity)
    print("Total number of reviews:", len(tk_reviews))
    lens = [len(article) for article in tk_reviews]
    print("Average review length:", np.mean(lens))
    print("Maximun review length:", np.max(lens))
    print("Minimun review length:", np.min(lens))
    print("Standard deviation of review length:", np.std(lens))

stats_print(df_desc_token)


Vocabulary size:  9834
Total number of tokens:  186952
Lexical diversity:  0.052601737344345076
Total number of reviews: 776
Average review length: 240.91752577319588
Maximun review length: 815
Minimun review length: 13
Standard deviation of review length: 124.97750685071483


#### 1.4 Remove words with length less than 2

In [13]:
df_desc_token.head()

0    [senior, sales, negotiator, location, shoredit...
1    [previous, experience, in, a, similar, role, i...
2    [graduate, sales, engineer, sector, engineerin...
3    [job, title, sales, representative, lead, gene...
4    [search, recruitment, consultant, media, and, ...
Name: Description, dtype: object

In [14]:
# for each token in each row, count the length of the token and save as a new df
df_desc_token_len = df_desc_token.apply(lambda x: [len(token) for token in x])
df_desc_token_len.head()

0    [6, 5, 10, 8, 10, 6, 9, 2, 10, 3, 1, 3, 9, 7, ...
1    [8, 10, 2, 1, 7, 4, 2, 9, 2, 4, 2, 6, 6, 7, 6,...
2    [8, 5, 8, 6, 11, 10, 8, 5, 4, 6, 5, 6, 4, 5, 4...
3    [3, 5, 5, 14, 4, 9, 8, 10, 6, 6, 8, 2, 5, 3, 1...
4    [6, 11, 10, 5, 3, 10, 8, 5, 6, 5, 1, 9, 8, 10,...
Name: Description, dtype: object

In [15]:
df_desc_token_clean = df_desc_token.apply(lambda x: [token for token in x if len(token) > 1])
df_desc_token_clean.head()

0    [senior, sales, negotiator, location, shoredit...
1    [previous, experience, in, similar, role, is, ...
2    [graduate, sales, engineer, sector, engineerin...
3    [job, title, sales, representative, lead, gene...
4    [search, recruitment, consultant, media, and, ...
Name: Description, dtype: object

#### 1.5 Remove stop words

In [16]:
# read in stop words document
stop_words = pd.read_csv('stopwords_en.txt', header=None, names=['stopword'])
stop_words.head()

,stopword
0,a
1,a's
2,able
3,about
4,above


In [17]:
# remove words that are a stop word
df_desc_token_clean = df_desc_token_clean.apply(lambda x: [token for token in x if token not in stop_words['stopword'].values])
df_desc_token_clean.head()

0    [senior, sales, negotiator, location, shoredit...
1    [previous, experience, similar, role, essentia...
2    [graduate, sales, engineer, sector, engineerin...
3    [job, title, sales, representative, lead, gene...
4    [search, recruitment, consultant, media, techn...
Name: Description, dtype: object

In [18]:
len(df_desc_token_clean)

776

#### 1.6 Remove words that appear only once in the document collection, based on term frequency

Note: Can also use hapaxes() function in W8 lab. Cross check this.

In [19]:
# take all terms across the whole df and put into a list
all_terms = list(chain.from_iterable(df_desc_token_clean))
# the above code can be interpreted as:
# for each list in df_desc_token_clean, chain.from_iterable will chain all the lists together
# list() will convert the chain object into a list
# the result is a list of all the terms in the df
# the chain.from_iterable() function is from the itertools library
# the itertools library is a collection of tools for handling iterators
# an iterator is an object that contains a countable number of values
# the chain() function takes several iterators as arguments and returns a single iterator
# the chain.from_iterable() function takes a single iterator as an argument and returns a single iterator

# count the frequency of each term
term_freq = pd.Series(all_terms).value_counts()
term_freq.head()

experience    1276
sales         1030
role           946
work           861
business       832
dtype: int64

In [20]:
term_freq.tail()

tempered        1
occassionaly    1
excellant       1
nuhalch         1
wiling          1
dtype: int64

In [21]:
term_freq.describe()

count   9404.000000
mean      11.395257
std       43.438569
min        1.000000
25%        1.000000
50%        2.000000
75%        6.000000
max     1276.000000
dtype: float64

In [22]:
# for words that only appear once in term_freq, remove them from df_desc_token_clean
df_desc_token_clean = df_desc_token_clean.apply(lambda x: [token for token in x if term_freq[token] > 1])
df_desc_token_clean.head()

0    [senior, sales, negotiator, location, shoredit...
1    [previous, experience, similar, role, essentia...
2    [graduate, sales, engineer, sector, engineerin...
3    [job, title, sales, representative, lead, gene...
4    [search, recruitment, consultant, media, techn...
Name: Description, dtype: object

In [23]:
# check that words with only 1 occurence have been removed
all_terms = list(chain.from_iterable(df_desc_token_clean))
term_freq = pd.Series(all_terms).value_counts()
term_freq.tail()

vibe             2
varying          2
preceptorship    2
compensation     2
possibilities    2
dtype: int64

In [24]:
len(term_freq)

5218

#### 1.7 Remove the top 50 most frequent words based on document frequency

This means taking looking at each sentence, getting a unique list of words in each sentence, flagging if that word occurs in each sentence (binary flag).
Then counting how many times the word appears across documents.

Note: Can use the FreqDist function to cross-check. See lab W8.

In [25]:
# for each sentence, get a unique list of words
df_desc_token_clean_unique = df_desc_token_clean.apply(lambda x: list(set(x)))
df_desc_token_clean_unique.head()

0    [negotiator, personalised, literacy, launderin...
1    [provide, previous, essential, customer, origi...
2    [bms, bmsgraduates, account, provide, managers...
3    [growing, aims, travel, difference, starting, ...
4    [scheme, originally, role, stable, technology,...
Name: Description, dtype: object

In [26]:
# get all the unique words along columns and the documents along the rows and count and count the number of times each word appears in each document
# Note: the words are now unique so we can use value_counts() as a binary flag to indicate whether the word appears in the document or not
df_token_doc_freq = df_desc_token_clean_unique.apply(lambda x: pd.Series(x).value_counts())
df_token_doc_freq.head()

negotiator  excellent  confident  outlook       uk  monthly  professional  \
0    1.000000   1.000000   1.000000 1.000000 1.000000 1.000000      1.000000   
1         NaN        NaN   1.000000      NaN      NaN      NaN           NaN   
2         NaN   1.000000        NaN      NaN 1.000000      NaN      1.000000   
3         NaN        NaN        NaN      NaN 1.000000      NaN      1.000000   
4         NaN   1.000000        NaN      NaN      NaN      NaN           NaN   

   driving      www   estate  relating  computer  closely   proven  company  \
0 1.000000 1.000000 1.000000  1.000000  1.000000 1.000000 1.000000 1.000000   
1      NaN 1.000000      NaN       NaN       NaN      NaN      NaN      NaN   
2 1.000000 1.000000      NaN       NaN       NaN      NaN      NaN 1.000000   
3      NaN 1.000000      NaN       NaN       NaN      NaN      NaN      NaN   
4      NaN 1.000000      NaN       NaN       NaN      NaN 1.000000 1.000000   

   targets    learn     high      act      met     join  written  financial  \
0 1.000000 1.000000 1.000000 1.000000 1.000000 1.000000 1.000000   1.000000   
1 1.000000      NaN      NaN      NaN      NaN      NaN      NaN        NaN   
2      NaN      NaN      NaN      NaN      NaN 1.000000      NaN        NaN   
3 1.000000      NaN      NaN      NaN      NaN      NaN 1.000000        NaN   
4      NaN      NaN 1.000000      NaN      NaN      NaN      NaN        NaN   

       job  requirements  reputation  brilliant   player  generate  working  \
0 1.000000      1.000000    1.000000   1.000000 1.000000  1.000000 1.000000   
1 1.000000           NaN         NaN        NaN      NaN       NaN      NaN   
2 1.000000           NaN         NaN        NaN      NaN       NaN      NaN   
3 1.000000           NaN    1.000000        NaN      NaN       NaN      NaN   
4 1.000000           NaN         NaN        NaN      NaN       NaN 1.000000   

   shoreditch    leads  friendly  depending   skills  demonstrating   office  \
0    1.000000 1.000000  1.000000   1.000000 1.000000       1.000000 1.000000   
1         NaN      NaN       NaN        NaN 1.000000            NaN      NaN   
2         NaN      NaN       NaN        NaN 1.000000            NaN      NaN   
3         NaN 1.000000       NaN        NaN 1.000000            NaN      NaN   
4         NaN      NaN       NaN        NaN      NaN            NaN 1.000000   

   established      ote  atmosphere  service  experience  understanding  \
0     1.000000 1.000000    1.000000 1.000000    1.000000       1.000000   
1          NaN      NaN         NaN      NaN    1.000000            NaN   
2     1.000000      NaN         NaN      NaN         NaN            NaN   
3          NaN 1.000000         NaN      NaN    1.000000            NaN   
4          NaN 1.000000         NaN      NaN    1.000000            NaN   

      oral  complying   posted  minimum  opportunities    ideas    basis  \
0 1.000000   1.000000 1.000000 1.000000       1.000000 1.000000 1.000000   
1      NaN        NaN 1.000000      NaN            NaN      NaN      NaN   
2      NaN        NaN 1.000000      NaN            NaN      NaN 1.000000   
3      NaN        NaN 1.000000 1.000000       1.000000      NaN      NaN   
4      NaN        NaN 1.000000      NaN            NaN      NaN      NaN   

     apply   senior  exceeded  succeed   whilst     type    fully  knowledge  \
0 1.000000 1.000000  1.000000 1.000000 1.000000 1.000000 1.000000   1.000000   
1      NaN      NaN       NaN      NaN      NaN      NaN      NaN   1.000000   
2      NaN      NaN       NaN 1.000000      NaN      NaN      NaN        NaN   
3 1.000000      NaN  1.000000      NaN      NaN 1.000000      NaN        NaN   
4      NaN 1.000000       NaN      NaN      NaN      NaN      NaN        NaN   

   communication  personalised  totaljobs  conversion  property    sales  \
0       1.000000      1.000000   1.000000    1.000000  1.000000 1.000000   
1            NaN           NaN   1.000000         NaN       NaN 1.000000   

In [27]:
# now, I can sum up every column to get the document frequency of each word
df_token_doc_freq = df_token_doc_freq.sum(axis=0)
df_token_doc_freq.head()

negotiator     6.000000
excellent    309.000000
confident     38.000000
outlook       18.000000
uk           316.000000
dtype: float64

In [28]:
# sort the df_token_doc_freq in descending order
df_token_doc_freq = df_token_doc_freq.sort_values(ascending=False)
df_token_doc_freq.head()

experience   586.000000
role         499.000000
work         453.000000
team         431.000000
working      407.000000
dtype: float64

In [29]:
# select the top 50 most frequent words by document frequency
df_token_doc_freq = df_token_doc_freq[:50]
df_token_doc_freq

experience    586.000000
role          499.000000
work          453.000000
team          431.000000
working       407.000000
skills        366.000000
client        358.000000
job           348.000000
company       343.000000
business      342.000000
uk            316.000000
excellent     309.000000
management    301.000000
based         287.000000
apply         286.000000
opportunity   280.000000
salary        270.000000
required      269.000000
successful    267.000000
support       261.000000
join          252.000000
candidate     248.000000
service       242.000000
knowledge     241.000000
development   235.000000
leading       234.000000
high          224.000000
cv            223.000000
manager       220.000000
www           220.000000
training      214.000000
strong        211.000000
sales         211.000000
provide       209.000000
including     209.000000
services      208.000000
ability       201.000000
contact       200.000000
position      199.000000
recruitment   196.000000


In [30]:
# remove these top 50 most frequent words from df_desc_token_clean
df_desc_token_clean = df_desc_token_clean.apply(lambda x: [token for token in x if token not in df_token_doc_freq.index])
df_desc_token_clean.head()

0    [senior, negotiator, location, shoreditch, dep...
1    [previous, similar, french, german, fluency, e...
2    [graduate, engineer, sector, engineering, comp...
3    [title, representative, lead, generator, locat...
4    [search, consultant, media, technology, bright...
Name: Description, dtype: object

In [31]:
df_desc_token_clean

0      [senior, negotiator, location, shoreditch, dep...
1      [previous, similar, french, german, fluency, e...
2      [graduate, engineer, sector, engineering, comp...
3      [title, representative, lead, generator, locat...
4      [search, consultant, media, technology, bright...
                             ...                        
771    [electrical, controls, engineer, audio, visual...
772    [field, engineer, mechanical, electrical, pump...
773    [buyer, part, time, vacancy, exists, experienc...
774    [duration, perm, contract, description, title,...
775    [prs, established, hvac, total, facility, buil...
Name: Description, Length: 776, dtype: object

#### 1.8 Save all job ad text and information in txt files, this will be used in Task 2 and 3

The output needs to be a vocab.txt file, with each word sorted in alphabetical order followed by a colon and an index, starting from 0. 

In [32]:
# convert df_desc_token_clean back into a unique list of words across all documents
all_terms_final = list(chain.from_iterable(df_desc_token_clean))
len(all_terms_final)

81205

In [33]:
# now unique the list and order by alphabetical order
all_terms_final = list(set(all_terms_final))
all_terms_final.sort()
len(all_terms_final)

5168

In [34]:
all_terms_final[:10]

['aap',
 'aaron',
 'aat',
 'abb',
 'abenefit',
 'aberdeen',
 'abi',
 'abilities',
 'abreast',
 'abroad']

In [35]:
# find example in raw document called df where the word ' aap ' appears
df_desc[df_desc['Description'].str.contains(' aap ')]

,Webindex,Description
171,72239537,a fantastic opportunity has arisen to work as ...


In [36]:
# check full text of the above example where webindex = 72239537
df_desc.iloc[0,:]

Webindex                                                72444142
Description    senior sales negotiator location : shoreditch ...
Name: 0, dtype: object

In [37]:
# to all_terms_final, create an index for each word, starting from 0 and incrementing by 1.
# the divider is :, formatted like this: word:index
# the join() function will join all the words in the list together, separated by the divider
# the enumerate() function will return an index value for each word in the list
# the index value will start from 0 and increment by 1
# the index value will be converted to a string
# the index value will be concatenated with the word
# the result is a list of words with an index value for each word
vocab_index = [word + ':' + str(index) for index, word in enumerate(all_terms_final)]
vocab_index[:10]

['aap:0',
 'aaron:1',
 'aat:2',
 'abb:3',
 'abenefit:4',
 'aberdeen:5',
 'abi:6',
 'abilities:7',
 'abreast:8',
 'abroad:9']

In [38]:
len(vocab_index)

5168

## Saving required outputs
Save the vocabulary, bigrams and job advertisment txt as per spectification.
- job advertisement text (raw read in file)
- tokenised job advertisement
- vocab.txt

In [39]:
# rename df_desc_token_clean description column to token_desc
#df_desc_token_clean = df_desc_token_clean.rename('token_desc')

In [40]:
df_desc_token_clean.head()

0    [senior, negotiator, location, shoreditch, dep...
1    [previous, similar, french, german, fluency, e...
2    [graduate, engineer, sector, engineering, comp...
3    [title, representative, lead, generator, locat...
4    [search, consultant, media, technology, bright...
Name: Description, dtype: object

In [41]:
df_desc_token_clean[0][0]

'senior'

In [42]:
# export raw data in df to csv
df.to_csv('raw_data.csv', index=False)

# export df_desc_token_clean to csv
#df_desc_token_clean.to_csv('cleaned_data.csv', index=False)

In [43]:
# export the vocab_index list to a text file
with open('vocab.txt', 'w') as file:
    file.write('\n'.join(vocab_index))


In [44]:
# export the cleaned data to a txt file with webindex as the unique identifier
#out_file = open("./cleaned_data.txt", 'w') # creates a txt file named './bbcNews.txt', open in write mode
#for article in df_desc_token_clean:
#    out_#file.write(' '.join(article) + '\n') # join the tokens in an article with space, and write the obtained string to the txt document
#out_file.close() # close the file


In [45]:
# Export the cleaned data to a txt file with webindex as the unique identifier
with open("./cleaned_data.txt", 'w') as out_file:
    for article in df_desc_token_clean:
        out_file.write(' '.join(article) + '\n')

## Summary

Task 1 has laid a solid foundation for our NLP Milestone 1 project, preparing a diverse collection of job advertisements for in-depth analysis. By meticulously cleaning and structuring the data, subsequent Tasks 2 and 3 can be executed effectively.

In Task 2, our cleaned and processed job advertisement descriptions will be used as inouts to generate various feature representations. These representations include count vectors and document embeddings based on selected language models, as well as TF-IDF weighted and unweighted versions. The output of Task 1, our vocabulary, will be instrumental in generating these representations, creating a seamless link between the tasks.

In Task 3, we will employ machine learning models to classify job advertisements into categories. The feature representations generated in Task 2 will play a crucial role in this classification process, allowing us to explore the performance of different language models. Additionally, we will investigate whether incorporating additional information, such as job titles, enhances classification accuracy.